<a href="https://colab.research.google.com/github/Stephensingletary/Image-Classification-For-Cross-Analysis-of-Chest-X-Ray/blob/main/final_Model_Patrick_Peters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5839%2F18613%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240523%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240523T170518Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4b25be70b2d403b00d1d2a2447e2a4425b1a5acf656fc1efead408167ed4849ecf7441c1b162d701e6a538f9b3a1494269abf1d5b66d92c4861e68a53b31ba9da4fee2813d0aa560aea69b7708e8316605fa05cf40e04db5feeb550e87bb156bf8a9fe00736067f43b468b3168562a0defe7b6d05c3c886c4b7902aae6488161bc23100757a6a6a7dd5e51fe7439d12c6078bbd9a579415ccade44170e061d4a5a1c30c34a1fb8a3a134bbafefec1886200595644e4cb601c197cfca447ff4a20fa34c3c4059280ae02dfc10e47f5f103522c5fd925d25ccfdc25322a30aa523b5da3305cc34ec02e99b6ff0d32ffb66fc14faf0217e928d0c7778b71a0b8b1e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 45096150231 bytes downloaded
Downloaded and uncompressed: data
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.densenet import DenseNet121

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential

from tensorflow.keras.models import load_model
from glob import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import tensorflow as tf

In [ ]:
entry_df = pd.read_csv("../input/data/Data_Entry_2017.csv")

def CreateTargetLabel(x):
    if x == "No Finding":
        return 0
    else:
        return 1
entry_df["target"] = entry_df["Finding Labels"].map(lambda x: CreateTargetLabel(x))
df = entry_df[["Image Index","target"]]

img_path = glob('/kaggle/input/data/images*/images*/*.png')
full_img_paths = {os.path.basename(x): x for x in img_path}
df['full_path'] = df['Image Index'].map(full_img_paths.get)
df = df.dropna(axis=0)


df = df.iloc[:20000]
print(df.target.value_counts())

target
0    11716
1     8284
Name: count, dtype: int64


<ipython-input-3-00c86f50c398>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_path'] = df['Image Index'].map(full_img_paths.get)


In [ ]:
def CreateBalancedDataframe(_df,samples_n=8000):
    target_y = _df[_df.target==1].reset_index().iloc[:samples_n]
    target_n = _df[_df.target==0].reset_index().iloc[:samples_n]
    return pd.concat([target_y,target_n])

df = CreateBalancedDataframe(df)
print(len(df))
print(df.target.value_counts())
df["target"] = df["target"].map(lambda x: "y" if x==1 else "n")

16000
target
1    8000
0    8000
Name: count, dtype: int64


In [ ]:
train_set, test_set = train_test_split(df, test_size = 0.2)

# quick check to see that the training and test set were split properly
print('training set - # of observations: ', len(train_set))
print('test set - # of observations): ', len(test_set))
print('prior, full data set - # of observations): ', len(df))

training set - # of observations:  12800
test set - # of observations):  3200
prior, full data set - # of observations):  16000


In [ ]:

data_gen = ImageDataGenerator(rescale=1/255.0,samplewise_center=True,samplewise_std_normalization= True)

train_gen = data_gen.flow_from_dataframe(train_set,batch_size=128,x_col="full_path",y_col="target",target_size=(224,224),class_mode="binary",color_mode="rgb",shuffle=True)
test_gen = data_gen.flow_from_dataframe(test_set,batch_size=128,x_col="full_path",y_col="target",target_size=(224,224),class_mode="binary",color_mode="rgb",shuffle=True)

Found 12800 validated image filenames belonging to 2 classes.
Found 3200 validated image filenames belonging to 2 classes.


In [ ]:
def build_model():
    model_1 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in model_1.layers:
        layer.trainable = False

    model_2 = DenseNet121(include_top=False,input_shape=(224,224,3))
    for layer in model_2.layers:
        layer.trainable=False

    input_layer = tf.keras.layers.Input(shape=(224,224, 3))
    out1 = model_1(input_layer)
    out2 = model_2(input_layer)

    fc1 = Flatten()(out1)
    fc2 = Flatten()(out2)
    fc_concat = tf.keras.layers.Concatenate()([fc1, fc2])
    fc3 = Dense(1024,activation="relu")(fc_concat)
    drop = Dropout(0.45)(fc3)
    fc_final = Dense(512,activation="relu")(drop)

    out = Dense(1,activation="sigmoid")(fc_final)
    return Model(inputs=input_layer,outputs=out)




hybrid_model = build_model()
loss = tf.keras.losses.BinaryCrossentropy()
hybrid_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])


29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
hybrid_model.fit(train_gen,validation_data=test_gen,epochs=10,verbose=1,steps_per_epoch=32,validation_steps=16)

Epoch 1/10
32/32 [==============================] - 134s 4s/step - loss: 18.5947 - accuracy: 0.4980 - val_loss: 1.0740 - val_accuracy: 0.4927
Epoch 2/10
32/32 [==============================] - 108s 3s/step - loss: 1.2218 - accuracy: 0.5376 - val_loss: 0.6636 - val_accuracy: 0.6113
Epoch 3/10
32/32 [==============================] - 104s 3s/step - loss: 0.7230 - accuracy: 0.5767 - val_loss: 0.6576 - val_accuracy: 0.6221
Epoch 4/10
32/32 [==============================] - 101s 3s/step - loss: 0.6406 - accuracy: 0.6382 - val_loss: 0.6477 - val_accuracy: 0.6309
Epoch 5/10
32/32 [==============================] - 101s 3s/step - loss: 0.6588 - accuracy: 0.6211 - val_loss: 0.6652 - val_accuracy: 0.6030
Epoch 6/10
32/32 [==============================] - 100s 3s/step - loss: 0.6534 - accuracy: 0.6160 - val_loss: 0.6487 - val_accuracy: 0.6470
Epoch 7/10
32/32 [==============================] - 100s 3s/step - loss: 0.6470 - accuracy: 0.6279 - val_loss: 0.6355 - val_accuracy: 0.6602
Epoch 8/10
3